In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:
def add_date_time(df):
    dts = []
    for idx in df.index:
        dts.append( datetime(
            df.loc[idx, 'year'],
            df.loc[idx, 'month'],
            df.loc[idx, 'day'],
            df.loc[idx, 'hour']-1, 
            0 # minutes
        ))
    df['date_time'] = dts
    return df

In [ ]:
demands = {
    'ERCOT' : ['../Input_Data/ReliabilityPaper/ERCOT_mem_1998-2019.csv', 0, 2003],
    'PJM' : ['../Input_Data/ReliabilityPaper/PJM_mem_1993-2019.csv', 0, 2006],
    'NYISO' : ['../Input_Data/ReliabilityPaper/NYISO_demand_unnormalized.csv', 5, 2004],
}

dfs = {}
for k, v in demands.items():
    print(k, v)
    df = pd.read_csv(v[0], header=v[1])
    print(len(df.index))
    df = df[ (df['year'] >= v[2]) & (df['year'] <= 2019) ]
    df = df.reset_index()
    df = add_date_time(df)
    print(len(df.index))
    dfs[k] = df.reset_index()
    print(dfs[k].head())
    print(dfs[k].tail())

In [ ]:
def fit(k, v):
    matplotlib.rcParams.update({'font.size': 20})
    plt.close()
    fix, ax = plt.subplots(figsize=(15,5))
    print(k, v)
    x = v.index
    y = v['demand (MW)']
    dt = v['date_time']
    coef = np.polyfit(x, y, 1)
    poly1d_fn = np.poly1d(coef) 
    # poly1d_fn is now a function which takes in x and returns an estimate for y
    print(coef)
    print(f"Delta 1 year: {coef[0]*8760}")
    print(f"Delta 1 year as % yr 1: {coef[0]*8760/coef[1]}")
    
    ax.plot(dt,y, 'y.')
    ax.plot(dt, poly1d_fn(x), '--k')
    
    ax.set_ylabel('load (MW)')
    ax.yaxis.set_major_formatter(matplotlib.ticker.StrMethodFormatter('{x:,.0f}'))
    ax.xaxis.set_major_locator(mdates.YearLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    ax.xaxis.set_minor_locator(mdates.MonthLocator())
    plt.setp( ax.xaxis.get_majorticklabels(), rotation=45 )
    
    ax.set_ylim(0, ax.get_ylim()[1])
    plt.tight_layout()
    plt.savefig(f'raw_demand_{k}.pdf')
    plt.savefig(f'raw_demand_{k}.png')

for k, v in dfs.items():
    print(f"\n\n{k}")
    fit(k, v)
    
    print(f"Mean 2019 demand: {np.mean(v.loc[ (v['year']==2019), 'demand (MW)'])}")